In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn as sk

In [2]:
traintest = pd.read_pickle('./traintest.p')

train = traintest[traintest.date < '2015-11-01'].drop(['sales','item_cnt_day','ID'], axis=1)
train['year'] = train.date.dt.year
train['month'] = train.date.dt.month
train['day'] = train.date.dt.day
train.drop('date',axis=1, inplace=True)
target = traintest[traintest.date < '2015-11-01']['item_cnt_day']
test = traintest[traintest.date >= '2015-11-01'].drop(['sales','item_cnt_day'],axis=1)
ID = test.ID
test.drop('ID', axis=1, inplace=True)
test['year'] = test.date.dt.year
test['month'] = test.date.dt.month
test['day'] = test.date.dt.day
test.drop('date',axis=1, inplace=True)

In [7]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import multiprocessing


xgb_model = xgb.XGBRegressor(n_jobs=multiprocessing.cpu_count() // 2)
#xgb_model = xgb.XGBRegressor(tree_method = 'gpu_hist', objective = 'reg:squarederror')
clf = GridSearchCV(xgb_model, {'max_depth': [2,4,6],
                                'n_estimators': [5,10,25]}, verbose=1,
                                
                    n_jobs=2)
clf.fit(train, target)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


KeyboardInterrupt: 

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

{'max_depth': 2, 'n_estimators': 50}
-0.03945919470759522


In [ ]:
test['item_cnt_day'] = clf.predict(test)
test['ID'] = ID

In [ ]:
submission = test.groupby(['ID']).sum().reset_index().sort_values('ID')[['ID','item_cnt_day']].astype({'ID':'int','item_cnt_day':'float64'})
submission.rename({'item_cnt_day': 'item_cnt_month'}, inplace=True, axis=1)
submission.to_csv('submission.csv', index= False)
submission

,ID,item_cnt_month
0,0,32.880676
1,1,2638.645752
2,2,33.046658
3,3,32.553474
4,4,2664.508301
...,...,...
214195,214195,32.102024
214196,214196,2690.069092
214197,214197,30.992195
214198,214198,2689.664795


In [ ]:
test

,shop_id,item_id,item_price,item_category_id,ends_with_9,ends_with_49,ends_with_99,item_price_below_q99,item_price_below_q95,item_price_below_q50,...,category_top5_cnt,category_top10_cnt,category_top20_cnt,weekday,holiday,year,month,day,item_cnt_day,ID
0,5,5037,749.5,19,True,True,False,True,True,False,...,False,False,False,6,False,2015,11,1,1.099847,0.0
1,5,5037,749.5,19,True,True,False,True,True,False,...,False,False,False,0,False,2015,11,2,1.074403,0.0
2,5,5037,749.5,19,True,True,False,True,True,False,...,False,False,False,1,False,2015,11,3,1.099847,0.0
3,5,5037,749.5,19,True,True,False,True,True,False,...,False,False,False,2,False,2015,11,4,1.099847,0.0
4,5,5037,749.5,19,True,True,False,True,True,False,...,False,False,False,3,False,2015,11,5,1.099847,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6425995,45,969,549.0,37,True,True,False,True,True,False,...,False,False,False,3,False,2015,11,26,1.132159,214199.0
6425996,45,969,549.0,37,True,True,False,True,True,False,...,False,False,False,4,False,2015,11,27,1.132159,214199.0
6425997,45,969,549.0,37,True,True,False,True,True,False,...,False,False,False,5,False,2015,11,28,1.132159,214199.0
6425998,45,969,549.0,37,True,True,False,True,True,False,...,False,False,False,6,False,2015,11,29,1.138401,214199.0


In [ ]:
kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission.csv -m "Message"